# Reactant demo

In [1]:
using Revise
using Tenet
using EinExprs
using Reactant
using AbstractTrees
using Random
using LinearAlgebra
using Enzyme
using Chairmarks
BLAS.set_num_threads(1)

MethodError(

2024-06-18 11:55:06.419858: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Core.convert, (AbstractString, Cstring(0x00000000010ece00)), 0x0000000000007b10)


In [2]:
Random.seed!(0)
tn = rand(TensorNetwork, 15, 3; dim=(16,16))

TensorNetwork (#tensors=15, #inds=22)

In [3]:
path = einexpr(tn; optimizer=Exhaustive())

SizedEinExpr{Symbol}(EinExpr{Symbol}(Symbol[], EinExpr{Symbol}[EinExpr{Symbol}([:R], EinExpr{Symbol}[EinExpr{Symbol}([:K], EinExpr{Symbol}[]), EinExpr{Symbol}([:R, :K], EinExpr{Symbol}[])]), EinExpr{Symbol}([:R], EinExpr{Symbol}[EinExpr{Symbol}([:G, :N, :R], EinExpr{Symbol}[]), EinExpr{Symbol}([:G, :N], EinExpr{Symbol}[EinExpr{Symbol}([:H, :Q, :N], EinExpr{Symbol}[]), EinExpr{Symbol}([:H, :Q, :G], EinExpr{Symbol}[EinExpr{Symbol}([:C, :F, :Q], EinExpr{Symbol}[]), EinExpr{Symbol}([:H, :C, :F, :G], EinExpr{Symbol}[EinExpr{Symbol}([:D, :H, :O, :C], EinExpr{Symbol}[]), EinExpr{Symbol}([:D, :O, :F, :G], EinExpr{Symbol}[EinExpr{Symbol}([:B, :D, :F, :J], EinExpr{Symbol}[EinExpr{Symbol}([:B, :D, :F, :V], EinExpr{Symbol}[]), EinExpr{Symbol}([:V, :J], EinExpr{Symbol}[EinExpr{Symbol}([:P, :J], EinExpr{Symbol}[]), EinExpr{Symbol}([:V, :P], EinExpr{Symbol}[EinExpr{Symbol}([:E], EinExpr{Symbol}[]), EinExpr{Symbol}([:E, :V, :P], EinExpr{Symbol}[])])])]), EinExpr{Symbol}([:B, :O, :G, :J], EinExpr{Symbo

In [4]:
maximum(ndims, Branches(path))

4

In [5]:
maximum(length, Branches(path))

65536

In [6]:
log10(mapreduce(flops, +, Branches(path)))

7.940067535122738678444270654298377812457104636982098412933015342439109112833246

In [7]:
expr = Tenet.codegen(Val(:outplace), path.path)

quote
    #= /home/bsc/bsc021106/Tenet.jl/src/Compiler.jl:92 =#
    function var"##contract_compiled#225"(ssa1::Tensor, ssa2::Tensor, ssa3::Tensor, ssa4::Tensor, ssa5::Tensor, ssa6::Tensor, ssa7::Tensor, ssa8::Tensor, ssa9::Tensor, ssa10::Tensor, ssa11::Tensor, ssa12::Tensor, ssa13::Tensor, ssa14::Tensor, ssa15::Tensor)
        #= /home/bsc/bsc021106/Tenet.jl/src/Compiler.jl:92 =#
        #= /home/bsc/bsc021106/Tenet.jl/src/Compiler.jl:93 =#
        ssa16 = contract(ssa1, ssa2)
        ssa17 = contract(ssa9, ssa10)
        ssa18 = contract(ssa8, ssa17)
        ssa19 = contract(ssa7, ssa18)
        ssa20 = contract(ssa14, ssa15)
        ssa21 = contract(ssa13, ssa20)
        ssa22 = contract(ssa12, ssa21)
        ssa23 = contract(ssa11, ssa22)
        ssa24 = contract(ssa19, ssa23)
        ssa25 = contract(ssa6, ssa24)
        ssa26 = contract(ssa5, ssa25)
        ssa27 = contract(ssa4, ssa26)
        ssa28 = contract(ssa3, ssa27)
        ssa29 = contract(ssa16, ssa28)
        #= /home/

In [8]:
f = @eval $expr

##contract_compiled#225 (generic function with 2 methods)

In [9]:
ts = [tn[head(leaf)...] for leaf in Leaves(path)];

In [10]:
# warm up
f(ts...)

@time f(ts...)

  0.006664 seconds (2.79 k allocations: 9.386 MiB)


0-dimensional Tensor{Float64, 0, Array{Float64, 0}}:
5.991293430814615e21

In [11]:
ts′ = [Tensor(Reactant.ConcreteRArray(parent(tensor)), inds(tensor)) for tensor in ts];
g = Reactant.compile(f, Tuple(ts′))

#109 (generic function with 1 method)

In [12]:
# warm up
g(ts′...)

@time x = g(ts′...)

  0.002995 seconds (6 allocations: 208 bytes)


0-dimensional Tensor{Float64, 0, Reactant.ConcreteRArray{Float64, (), 0}}:
5.991293430814615e21

In [13]:
expr = Tenet.codegen(Val(:grad), path.path)

quote
    #= /home/bsc/bsc021106/Tenet.jl/src/Compiler.jl:173 =#
    function var"##∇contract#229"(ssa1::Tensor, ssa2::Tensor, ssa3::Tensor, ssa4::Tensor, ssa5::Tensor, ssa6::Tensor, ssa7::Tensor, ssa8::Tensor, ssa9::Tensor, ssa10::Tensor, ssa11::Tensor, ssa12::Tensor, ssa13::Tensor, ssa14::Tensor, ssa15::Tensor)
        #= /home/bsc/bsc021106/Tenet.jl/src/Compiler.jl:173 =#
        #= /home/bsc/bsc021106/Tenet.jl/src/Compiler.jl:175 =#
        ssa16 = contract(ssa1, ssa2)
        ssa17 = contract(ssa9, ssa10)
        ssa18 = contract(ssa8, ssa17)
        ssa19 = contract(ssa7, ssa18)
        ssa20 = contract(ssa14, ssa15)
        ssa21 = contract(ssa13, ssa20)
        ssa22 = contract(ssa12, ssa21)
        ssa23 = contract(ssa11, ssa22)
        ssa24 = contract(ssa19, ssa23)
        ssa25 = contract(ssa6, ssa24)
        ssa26 = contract(ssa5, ssa25)
        ssa27 = contract(ssa4, ssa26)
        ssa28 = contract(ssa3, ssa27)
        ssa29 = contract(ssa16, ssa28)
        #= /home/bsc/b

In [14]:
∇f = @eval $expr

##∇contract#229 (generic function with 2 methods)

In [20]:
@time ∇f(ts...)

  0.025264 seconds (8.26 k allocations: 30.864 MiB, 31.47% gc time)


(primal = fill(5.991293430814615e21), grad = ([8.59862507557731e20, 9.854553258399852e20, 8.689630432571684e20, 8.560880733756893e20, 1.000969369150795e21, 9.34727678010833e20, 9.540873738359265e20, 7.844058750911498e20, 7.837209035462038e20, 9.153231844847093e20, 7.154143930644968e20, 5.977852489070245e20, 1.0908543832455945e21, 1.080171724330224e21, 7.434074738655267e20, 7.227929321768543e20], [8.058707958736981e19 1.3212083508969722e19 … 7.22915465164192e19 6.038729129053087e18; 7.759345205241892e19 1.2721284522469149e19 … 6.960607925164375e19 5.814404015282906e18; … ; 8.004998574954291e19 1.3124028095200463e19 … 7.1809740445771735e19 5.99848242672652e18; 8.253612772819596e19 1.3531625883894465e19 … 7.403996208200022e19 6.184779511350081e18], [3.161450367837677e18 2.985863553327797e18 … 3.088894378172048e18 3.4658326178167137e18; 3.21233053190811e18 3.033917179870332e18 … 3.1385246325301274e18 3.521630883515899e18; … ; 3.185828102546754e18 3.0088571999220014e18 … 3.1125970977224443e

In [25]:
tf = @b ∇f(ts...)

14.351 ms (8272 allocs: 30.865 MiB)

In [17]:
# ∇g = Reactant.compile(∇f, Tuple(ts′))
function grad(x...)
    dx = Enzyme.make_zero.(x)
    Enzyme.autodiff(Reverse, f, Active, Duplicated.(x,dx)...)
    return dx
end
∇g = Reactant.compile(grad, Tuple(ts′))

#111 (generic function with 1 method)

In [18]:
∇g(ts′...)

@time ∇g(ts′...)

  0.008577 seconds (91 allocations: 3.703 KiB)


([8.598625075577312e20, 9.854553258399851e20, 8.689630432571684e20, 8.560880733756895e20, 1.000969369150795e21, 9.347276780108328e20, 9.540873738359266e20, 7.844058750911498e20, 7.837209035462039e20, 9.153231844847094e20, 7.154143930644966e20, 5.977852489070245e20, 1.0908543832455943e21, 1.0801717243302239e21, 7.434074738655269e20, 7.227929321768544e20], [8.058707958736986e19 1.3212083508969732e19 … 7.229154651641926e19 6.038729129053091e18; 7.759345205241889e19 1.2721284522469143e19 … 6.960607925164373e19 5.814404015282904e18; … ; 8.004998574954293e19 1.3124028095200467e19 … 7.180974044577175e19 5.998482426726521e18; 8.2536127728196e19 1.3531625883894469e19 … 7.403996208200024e19 6.184779511350083e18], [3.1614503678376786e18 2.9858635533277993e18 … 3.0888943781720484e18 3.465832617816712e18; 3.212330531908107e18 3.0339171798703324e18 … 3.1385246325301263e18 3.5216308835158984e18; … ; 3.1858281025467546e18 3.0088571999220004e18 … 3.1125970977224453e18 3.4926363521079286e18; 3.157862255

In [21]:
@b ∇g(ts′...)

8.227 ms (99 allocs: 4.359 KiB)

In [22]:
# ∇g = Reactant.compile(∇f, Tuple(ts′))
function regrad(x...)
    dx = Enzyme.make_zero.(x)
    res = Enzyme.autodiff(ReverseWithPrimal, f, Active, Duplicated.(x,dx)...)
    return (res, dx)
end
re∇g = Reactant.compile(regrad, Tuple(ts′))

#113 (generic function with 1 method)

In [23]:
re∇g(ts′...)

@time re∇g(ts′...)

  0.008804 seconds (99 allocations: 4.203 KiB)


(((nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing), fill(5.991293430814615e21)), ([8.598625075577312e20, 9.854553258399851e20, 8.689630432571684e20, 8.560880733756895e20, 1.000969369150795e21, 9.347276780108328e20, 9.540873738359266e20, 7.844058750911498e20, 7.837209035462039e20, 9.153231844847094e20, 7.154143930644966e20, 5.977852489070245e20, 1.0908543832455943e21, 1.0801717243302239e21, 7.434074738655269e20, 7.227929321768544e20], [8.058707958736986e19 1.3212083508969732e19 … 7.229154651641926e19 6.038729129053091e18; 7.759345205241889e19 1.2721284522469143e19 … 6.960607925164373e19 5.814404015282904e18; … ; 8.004998574954293e19 1.3124028095200467e19 … 7.180974044577175e19 5.998482426726521e18; 8.2536127728196e19 1.3531625883894469e19 … 7.403996208200024e19 6.184779511350083e18], [3.1614503678376786e18 2.9858635533277993e18 … 3.0888943781720484e18 3.465832617816712e18; 3.212330531908107e18 3.03391

In [28]:
tg = @b re∇g(ts′...)

8.481 ms (107 allocs: 4.891 KiB)

In [31]:
propertynames(tf)

(:evals, :time, :allocs, :bytes, :gc_fraction, :compile_fraction, :recompile_fraction, :warmup, :checksum)

In [39]:
@show speedup = tf.time / tg.time

speedup = tf.time / tg.time = 1.6920500933703184


1.6920500933703184